# Assignment 1.2: Word2vec preprocessing (20 points)

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. 
Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). 
For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or 
handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make **Batcher** class which returns two numpy tensors with word indices. It should be possible 
to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below 
can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size, 2*window_size)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement 
only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. 
The result of your work should represent that your batch has a proper structure (right shapes) and content 
(words should be from one context, not some random indices). To show that, translate indices back to words and 
print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']
```

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import gensim.downloader as api

def load_text8():
    dataset = api.load("text8")
    with open('text8.txt', 'w+') as f:
        f.write('\n'.join([' '.join(words) for words in dataset]))
load_text8()

[==================================================] 100.0% 31.6/31.6MB downloaded


Код находится в файле batcher

In [2]:
from batcher import Batcher

In [5]:
batcher = Batcher('text8.txt', model_type='cbow', min_count=5)

Making a dictionary of words: 1701it [01:18, 21.71it/s]


In [7]:
print('Количество слов в словаре', batcher.vocab_size)

Количество слов в словаре 63638


batcher.word_count - словарь пар слово - количество раз, которое оно встретилось в датасете

batcher.word2index, batcher.index2word- словари для numericalization

In [9]:
next(iter(batcher.generate_batches(batch_size=4)))

{'x_positive': tensor([[ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10],
         [ 1,  2,  3,  4,  5,  7,  8,  9, 10, 11],
         [ 2,  3,  4,  5,  6,  8,  9, 10, 11, 12],
         [ 3,  4,  5,  6,  7,  9, 10, 11, 12, 13]]), 'y': tensor([5, 6, 7, 8])}

Если параметр ```num_negative_samples = 0```, то генерируются данные без негативного сэмплирования

In [10]:
next(iter(batcher.generate_batches(batch_size=4, num_negative_samples=10)))

{'x_positive': tensor([[ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10],
         [ 1,  2,  3,  4,  5,  7,  8,  9, 10, 11],
         [ 2,  3,  4,  5,  6,  8,  9, 10, 11, 12],
         [ 3,  4,  5,  6,  7,  9, 10, 11, 12, 13]]),
 'y': tensor([5, 6, 7, 8]),
 'x_negative': tensor([[24472, 44900,    74, 13456, 13760,   167,     5, 51825,   197,   252],
         [ 2157, 39404,  2212, 43144, 52452,   178,    26,  7812,  6354,   230],
         [    5, 10369,   254, 27561, 24030, 10243, 31379,  8222,   233, 18165],
         [51521,  2107, 13908,  1934,   200,   291, 23002,  6244,   340,  6754]])}

Пример с негативным сэмплированием